In [1]:
import os
print(os.getcwd())

C:\Users\Saeah Go\Desktop\CS540 Deep Learning


# Q1 a

In [2]:
import yfinance as yf
import time
import numpy as np
import pandas as pd
import shutil
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.utils as vutils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Ensure reproducibility
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 40  

# CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          num_workers=0,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         num_workers=0,
                         shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified


In [4]:
# CNN Model
class LeNetCustom(nn.Module):
    def __init__(self, num_classes, activation_fn):
        super(LeNetCustom, self).__init__()
        self.activation_fn = activation_fn

        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1),
            self.activation_fn(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            self.activation_fn(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            self.activation_fn(),
            nn.Linear(120, 84),
            self.activation_fn(),
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits
    
    def get_feature_maps(self, x):
        # Pass input through the feature extraction layers
        with torch.no_grad():
            feature_maps = self.features(x)
        return feature_maps

In [5]:
# Loss computation function
def compute_loss(logits, targets, loss_fn):
    if isinstance(loss_fn, nn.MSELoss):
        targets_one_hot = F.one_hot(targets, num_classes=num_classes).float()
        return loss_fn(logits, targets_one_hot)
    return loss_fn(logits, targets)

In [6]:
# Training function
def train_model(model, optimizer, loss_fn, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_pred = 0
    num_examples = 0
    
    for features, targets in train_loader:
        features, targets = features.to(device), targets.to(device)

        optimizer.zero_grad()
        logits = model(features)
        loss = compute_loss(logits, targets, loss_fn)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * targets.size(0)
        correct_pred += (logits.argmax(dim=1) == targets).sum().item()
        num_examples += targets.size(0)
        
    return running_loss / len(train_loader.dataset), (correct_pred / num_examples) * 100

In [7]:
# Evaluation function
def evaluate_model(model, loss_fn, data_loader, device):
    model.eval()
    running_loss = 0.0
    correct_pred = 0
    num_examples = 0
    
    with torch.no_grad():
        for features, targets in data_loader:
            features, targets = features.to(device), targets.to(device)
            logits = model(features)
            loss = compute_loss(logits, targets, loss_fn)

            running_loss += loss.item() * targets.size(0)
            correct_pred += (logits.argmax(dim=1) == targets).sum().item()
            num_examples += targets.size(0)

    return running_loss / len(data_loader.dataset), (correct_pred / num_examples) * 100

In [8]:
# Hyperparameter tuning function
def run_experiment(lr, activation_fn, loss_name, loss_fn):
    run_name = f"LR={lr}, Activation={activation_fn.__name__}, Loss={loss_name}"
    print(f"Initializing wandb run: {run_name}")

    with wandb.init(entity="saeah-portland-state-university",
                    project="cs540-programming2",
                    name=run_name,
                    config={"learning_rate": lr, "activation_function": activation_fn.__name__, "loss_function": loss_name, "epochs": epochs}):

        model = LeNetCustom(num_classes=num_classes, activation_fn=activation_fn).to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr) #, momentum=0.9)
        wandb.watch(model, log="all", log_freq=10)

        for epoch in range(epochs):
            train_loss, train_accuracy = train_model(model, optimizer, loss_fn, train_loader, device)
            test_loss, test_accuracy = evaluate_model(model, loss_fn, test_loader, device)

            # Every 10 epochs, print the stats
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%", flush=True)
            
            # Log metrics to wandb
            wandb.log({
                "train_loss": train_loss, 
                "train_accuracy": train_accuracy, 
                "test_loss": test_loss, 
                "test_accuracy": test_accuracy, 
                "epoch": epoch + 1})
        
        # Save the trained model
        model_filename = f"model_lr={lr}_activation={activation_fn.__name__}_loss={loss_name}.pth"
        model_path = os.path.join("saved_models", model_filename)
        os.makedirs("saved_models", exist_ok=True)
        torch.save(model.state_dict(), model_path)

        wandb_model_path = os.path.join(wandb.run.dir, os.path.basename(model_path))
        shutil.copy(model_path, wandb_model_path)
        
    wandb.finish()

In [9]:
# Hyperparameters
learning_rates = [0.1, 0.01, 0.001] 
activation_functions = [nn.Sigmoid, nn.Tanh] 
loss_functions = {"MSE": nn.MSELoss(), "CrossEntropy": nn.CrossEntropyLoss()} 

# Run experiments
for lr in learning_rates:
    for activation_fn in activation_functions:
        for loss_name, loss_fn in loss_functions.items():
            run_experiment(lr, activation_fn, loss_name, loss_fn)

Initializing wandb run: LR=0.1, Activation=Sigmoid, Loss=MSE


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: saeah (saeah-portland-state-university). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 10/40 - Train Loss: 0.0902, Train Acc: 10.13% | Test Loss: 0.0904, Test Acc: 10.00%
Epoch 20/40 - Train Loss: 0.0902, Train Acc: 10.03% | Test Loss: 0.0903, Test Acc: 10.00%
Epoch 30/40 - Train Loss: 0.0902, Train Acc: 9.81% | Test Loss: 0.0903, Test Acc: 10.00%
Epoch 40/40 - Train Loss: 0.0902, Train Acc: 10.18% | Test Loss: 0.0903, Test Acc: 10.00%
Model saved: saved_models\model_lr=0.1_activation=Sigmoid_loss=MSE.pth


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▅▁▂▃▄▅▂▂▁▄▄▃▄▄▂▃▄▂▄▂▁▄▂▅▂▁▅▅▂▃▅▃█▇▅▄▃▃▂▄
train_accuracy,▆▂▆▂▃▄▄▄▂▅▃▆▃▂▂▃▁▆▂▄█▄▂▄▃▅▃▃▄▂▅▁▆▄▃▆▄▂▆▆
train_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▂▁▁▁▂▂▁▂▂▁▂▁▁▁
epoch,40
test_accuracy,10
test_loss,0.09033
train_accuracy,10.18
train_loss,0.09019


Finishing wandb run...
Initializing wandb run: LR=0.1, Activation=Sigmoid, Loss=CrossEntropy


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████
test_loss,█▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████
train_loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,40
test_accuracy,18.68
test_loss,0.08918
train_accuracy,18.636
train_loss,0.0892


Finishing wandb run...
Initializing wandb run: LR=0.001, Activation=Tanh, Loss=CrossEntropy


Epoch 10/40 - Train Loss: 2.2865, Train Acc: 11.74% | Test Loss: 2.2845, Test Acc: 11.67%
Epoch 20/40 - Train Loss: 2.1143, Train Acc: 19.53% | Test Loss: 2.1058, Test Acc: 19.66%
Epoch 30/40 - Train Loss: 2.0530, Train Acc: 22.96% | Test Loss: 2.0480, Test Acc: 23.59%
Epoch 40/40 - Train Loss: 2.0127, Train Acc: 25.51% | Test Loss: 2.0071, Test Acc: 25.73%
Model saved: saved_models\model_lr=0.001_activation=Tanh_loss=CrossEntropy.pth


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁▁▃▃▃▂▂▂▂▂▂▃▄▅▆▆▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██████
test_loss,██████████▇▇▇▇▆▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train_accuracy,▁▁▂▃▃▂▂▂▂▂▂▂▃▄▅▆▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██████
train_loss,██████████▇▇▇▇▇▆▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch,40
test_accuracy,25.73
test_loss,2.00707
train_accuracy,25.512
train_loss,2.01273


Finishing wandb run...


# Q1 b 
Visualize feature maps using wandb

In [10]:
# Load the Model
model_filename = "saved_models/model_lr=0.01_activation=Tanh_loss=CrossEntropy.pth"

# Initialize model with the same architecture
model = LeNetCustom(num_classes=10, activation_fn=nn.Tanh).to(device)

# Load the saved model weights
model.load_state_dict(torch.load(model_filename))
model.eval()  # Set model to evaluation mode

C:\Users\Saeah Go\AppData\Local\Temp\ipykernel_28836\2336507280.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_filename))


LeNetCustom(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Tanh()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [11]:
# Select 10 images for feature map visualization
sample_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
images, _ = next(iter(sample_loader))

# Move images to the same device as the model
images = images.to(device)

In [12]:
# Extract and visualize feature maps

# Initialize a new wandb run for visualization
wandb.init(
    entity="saeah-portland-state-university",
    project="cs540-programming2",
    name="Q1b_Feature_Map_Visualization"
)

# Get feature maps from the last convolutional layer
with torch.no_grad():
    feature_maps = model.get_feature_maps(images)  # Shape: (10, 16, 5, 5)

# Log feature maps to wandb
for i in range(10):  
    img_feature_maps = feature_maps[i]  # Shape: (16, 5, 5)

    img_feature_maps = torch.nn.functional.interpolate(img_feature_maps.unsqueeze(0), size=(5, 5), mode="bilinear", align_corners=False)
    img_feature_maps = img_feature_maps.squeeze(0)  

    # Convert feature maps into a 4x4 grid (16 channels)
    grid = vutils.make_grid(img_feature_maps.unsqueeze(1), nrow=4, normalize=True, scale_each=True)

    wandb.log({f"Feature Maps Image {i+1}": [wandb.Image(grid, caption=f"Feature Maps for Image {i+1}")]})

# Finish wandb run
wandb.finish()

# Q2 a 
Experiment with ReLU

In [13]:
# CNN Model
class LeNetCustom3x3(nn.Module):
    def __init__(self, num_classes, activation_fn):
        super(LeNetCustom3x3, self).__init__()
        self.activation_fn = activation_fn

        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=1),  # 3x3x3 kernels with padding -> 32x32x6
            self.activation_fn(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 16x16x6
            nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=1),  # 3x3x6 kernels with padding -> 16x16x16
            self.activation_fn(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 8x8x16
        )

        self.classifier = nn.Sequential(
            nn.Linear(16 * 8 * 8, 120),
            self.activation_fn(),
            nn.Linear(120, 84),
            self.activation_fn(),
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

In [14]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 40  
learning_rate = 0.001

# Initialize wandb
wandb.init(
    project="cs540-programming2",
    name="Q2_a_ReLU",
    config={
        "epochs": epochs,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "architecture": "LeNetCustom3x3",
        "optimizer": "SGD",
        "activation_function": "ReLU",
    },
)
config = wandb.config

# Model Training
model = LeNetCustom3x3(num_classes=num_classes, activation_fn=nn.ReLU).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #, momentum=0.9)
loss_fn = nn.CrossEntropyLoss() 

for epoch in range(epochs):
    train_loss, train_accuracy = train_model(model, optimizer, loss_fn, train_loader, device)
    test_loss, test_accuracy = evaluate_model(model, loss_fn, test_loader, device)

    wandb.log({
        "train_loss": train_loss,
        "train_accuracy": train_accuracy, 
        "test_loss": test_loss,
        "test_accuracy": test_accuracy,
        "epoch": epoch + 1
    })
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.2f}% | Test Loss: {test_loss:.4f} | Test Acc: {test_accuracy:.2f}%", flush=True)

wandb.finish()        

Epoch 10/40 | Train Loss: 2.3024 | Train Acc: 14.53% | Test Loss: 2.3022 | Test Acc: 14.27%
Epoch 20/40 | Train Loss: 2.3003 | Train Acc: 12.54% | Test Loss: 2.3001 | Test Acc: 12.11%
Epoch 30/40 | Train Loss: 2.2958 | Train Acc: 12.22% | Test Loss: 2.2952 | Test Acc: 12.18%
Epoch 40/40 | Train Loss: 2.2803 | Train Acc: 16.46% | Test Loss: 2.2782 | Test Acc: 16.35%


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁▁▁▂▄▅▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▄▄▅▆▆▇▇███
test_loss,█████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▃▃▂▁
train_accuracy,▁▁▁▂▃▅▅▆▅▆▅▅▅▅▅▅▅▄▄▄▄▃▃▂▃▃▃▃▃▃▄▄▅▅▆▇▇▇██
train_loss,████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▄▄▄▃▂▂▁
epoch,40
test_accuracy,16.35
test_loss,2.27818
train_accuracy,16.46
train_loss,2.28034


# Q3 a

Predict 51st sin(x) wave

In [15]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, sine=False):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sine = sine
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)  # Initialize hidden state
        out, _ = self.rnn(x, h0)  

        # Use the entire sequence or just the last output, based on the sine flag
        if self.sine:
            # Return the full sequence for the sine wave prediction
            return self.fc(out)
        else:
            # Only return the last output (for Dow Jones prediction)
            return self.fc(out[:, -1, :])  # Predict the last time step (101st value)

In [16]:
def to_dataloader(X, y, batch_size=32, unsqueeze_dim=None):
    if unsqueeze_dim is not None:
        X = X.unsqueeze(unsqueeze_dim)
        y = y.unsqueeze(unsqueeze_dim)
    dataset = torch.utils.data.TensorDataset(X, y)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [17]:
# Initialize Weights & Biases
wandb.init(project="cs540-programming2",
          name="Q3_a")

# Generate sine wave data
def generate_data(seq_length, num_samples):
    X = []
    y = []
    for i in range(num_samples):
        x = np.linspace(i * 2 * np.pi, (i + 1) * 2 * np.pi, seq_length + 1)
        sine_wave = np.sin(x)
        X.append(sine_wave[:-1])  # input sequence
        y.append(sine_wave[1:])   # target sequence
    return np.array(X), np.array(y)

# Generate dataset
seq_length = 50
num_samples = 1000
X, y = generate_data(seq_length, num_samples)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Split into training, validation, and test datasets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

train_loader = to_dataloader(X_train, y_train, unsqueeze_dim=2)
val_loader = to_dataloader(X_val, y_val, unsqueeze_dim=2)
test_loader = to_dataloader(X_test, y_test, batch_size=1, unsqueeze_dim=2)  # Batch size 1 for test

# Model parameters
input_size = 1
hidden_size = 20
output_size = 1
learning_rate = 0.001
epochs = 100

model = SimpleRNN(input_size, hidden_size, output_size,sine=True)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Log model parameters
wandb.config.update({
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "learning_rate": learning_rate,
    "epochs": epochs,
})

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    # Test loss (without updating model)
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            test_loss += loss.item()
    test_loss /= len(test_loader)  # Compute average test loss for this epoch

    # Log train, validation, and test loss per epoch
    wandb.log({
        "Train Loss": train_loss,
        "Validation Loss": val_loss,
        "Test Loss": test_loss,  # Now tracking test loss per epoch
        "Epoch": epoch + 1
    })

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Test Loss: {test_loss:.4f}", flush=True)

# Finish wandb run
wandb.finish()

Epoch [10/100], Train Loss: 0.0006, Val Loss: 0.0005, Test Loss: 0.0005
Epoch [20/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [30/100], Train Loss: 0.0001, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [40/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [50/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [60/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [70/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [80/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [90/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000
Epoch [100/100], Train Loss: 0.0000, Val Loss: 0.0000, Test Loss: 0.0000


Epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
Test Loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,█▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,100
Test Loss,0.0
Train Loss,0.0
Validation Loss,0.0


# Q3 b
Download dow jones without string values and predict 101st values after seeing 100 values

In [18]:
# Initialize Weights & Biases
wandb.init(project="cs540-programming2",
          name="Q3_b")

# Download Dow Jones data
def download_dow_jones_data():
    # Download historical data for Dow Jones Industrial Average (^DJI)
    data = yf.download("^DJI", start="2010-01-01", end="2023-01-01")
    data.to_csv("dow_jones.csv") # Save to CSV
    return data

# Load or download data
try:
    data = pd.read_csv("dow_jones.csv")
except FileNotFoundError:
    data = download_dow_jones_data()

# Drop rows with non-numerical values in the High column
data = data[pd.to_numeric(data["High"], errors="coerce").notnull()]

# Use the High column
high_prices = data["High"].values.reshape(-1, 1)

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
high_prices_scaled = scaler.fit_transform(high_prices)

# Create sequences
seq_length = 100
X, y = create_sequences(high_prices_scaled, seq_length)

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_loader = to_dataloader(X_train, y_train)
val_loader = to_dataloader(X_val, y_val)
test_loader = to_dataloader(X_test, y_test, batch_size=1) # Batch size 1 for test

# Model parameters
input_size = 1
hidden_size = 50
output_size = 1
learning_rate = 0.001
epochs = 100

model = SimpleRNN(input_size, hidden_size, output_size, sine=False)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

wandb.config.update({
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "learning_rate": learning_rate,
    "epochs": epochs,
})

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    
    # Test loss (without updating model)
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            test_loss += loss.item()
    test_loss /= len(test_loader)  # Compute average test loss for this epoch
   
    # Log train, validation, and test loss per epoch
    wandb.log({
        "Train Loss": train_loss,
        "Validation Loss": val_loss,
        "Test Loss": test_loss,  # Now tracking test loss per epoch
        "Epoch": epoch + 1
    })

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Test Loss: {test_loss:.4f}", flush=True)

# Inverse transform the predictions and targets
test_predictions = scaler.inverse_transform(np.array(test_predictions).reshape(-1, 1))
test_targets = scaler.inverse_transform(np.array(test_targets).reshape(-1, 1))

# Log predictions and targets to wandb as a table
table = wandb.Table(columns=["Sample", "True High Price", "Predicted High Price"])
for i in range(len(test_targets)):
    table.add_data(i, test_targets[i][0], test_predictions[i][0])

wandb.log({"Test Predictions vs. True Values": table})

# Finish the wandb 
wandb.finish()

Epoch [10/100], Train Loss: 0.0002, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [20/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [30/100], Train Loss: 0.0001, Val Loss: 0.0002, Test Loss: 0.0002
Epoch [40/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [50/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [60/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [70/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [80/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [90/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001
Epoch [100/100], Train Loss: 0.0001, Val Loss: 0.0001, Test Loss: 0.0001


Epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█████
Test Loss,▅▅▄▇▄▃▃▄▆▆█▅▃▃▂▇▃▃▃▃▂▃▂▃▃▂▃▃▂▂▄▂▁▂▂▁▁▁▁▂
Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,▇▆▄▆▃▃▃▃▃█▂▂▂▂▆▂▂▂▂▂▁▂▄▂▂▂▂▃▁▁▂▁▁▂▂▁▂▁▁▁
Epoch,100
Test Loss,7e-05
Train Loss,7e-05
Validation Loss,7e-05
